# Sum

In [3]:
import d2ltvm
import numpy as np
import tvm
from tvm import te

successd...



In [4]:
np.random.seed(0)
shape = (3,4)
dtype = 'float32'
a = np.random.normal(size=shape).astype(dtype)
a.sum(axis=1)

array([5.3838406, 1.6890113, 1.9056966], dtype=float32)

In [5]:
def sum_rows(a, b):
    n = len(b)
    for i in range(n):
        b[i] = sum(a[i,:])

b = np.empty(shape[0]).astype(dtype)
sum_rows(a, b)
b

array([5.3838406, 1.6890113, 1.9056966], dtype=float32)

In [6]:
n, m = te.var(name='n'), te.var(name='m')
A = te.placeholder((n, m), dtype=dtype, name='a')
j = te.reduce_axis((0, m), name='j')
B = te.compute((n,), lambda i: te.sum(A[i,j], axis=j), name='b')
s = te.create_schedule(B.op)
print(tvm.lower(s, [A, B], simple_mode=True).show())

None


In [7]:
mod = tvm.build(s, [A, B])
c = tvm.nd.array(np.empty(shape[0]).astype(dtype))
mod(tvm.nd.array(a), c)
np.testing.assert_equal(c.asnumpy(), b)

In [8]:
i = te.reduce_axis((0, n), name='i')
B = te.compute((), lambda : te.sum(A[i, j], axis=(i, j)), name='n')
s = te.create_schedule(B.op)
tvm.lower(s, [A, B], simple_mode=True).show()

In [9]:
mod = tvm.build(s, [A, B])
c = tvm.nd.array(np.empty((), dtype=dtype))
mod(tvm.nd.array(a), c)
np.testing.assert_equal(a.sum(), c.asnumpy())

# Commutative Reduction

In [10]:
def prob_rows(a, b):
    n, m = a.shape
    for i in range(n):
        b[i] = 1
        for j in range(m):
            b[i] = b[i] * a[i,j]

In [ ]:
# comp = lambda a, b: a * b
init = lambda  dtype: tvm.tir.const(1, dtype=dtype)
product = te.comm_reducer(comp, init)
n = te.var(name='n')
m = te.var(name='m')
A = te.placeholder((n, m), name='a')
k = te.reduce_axis((0, m), name='k')
B = te.compute((n,), lambda i: product(A[i, k], axis=k), name='b')
s = te.create_schedule(B.op)
tvm.lower(s, [A, B]).show()

In [19]:
mod = tvm.build(s, [A, B])
b = tvm.nd.array(np.empty(a.shape[0]).astype(dtype))
mod(tvm.nd.array(a), b)
np.testing.assert_equal(a.prod(axis=1), b.asnumpy())

# add demo

In [34]:
comp = lambda a, b: a + b
init = lambda dtype: tvm.tir.const(0, dtype=dtype)
product = te.comm_reducer(comp, init)
A = te.placeholder((n, m), dtype=dtype, name='a')
k = te.reduce_axis((0, m), name='k')
B = te.compute((n,), lambda i: product(A[i,k], axis=k), name='b')
s = te.create_schedule(B.op)
tvm.lower(s, [A, B]).show()

In [36]:
mod = tvm.build(s, [A, B, n, m])
shape = (4, 5)
a = np.random.normal(size=shape).astype(dtype)
b = np.empty(shape[0]).astype(dtype)
b = tvm.nd.array(b)
mod(tvm.nd.array(a), b, *shape)
np.testing.assert_equal(a.sum(axis=1), b.asnumpy())

# Summary
1.We can apply a reduction operator, e.g. te.sum over a reduction axis te.reduce_axis.

2.We can implement customized commutative reduction operators by te.comm_reducer